# LLamaIndex

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/llamaindex.ipynb)

Ce bloc-nore Jupyter démontre une application de Génération Augmentée de Récupération (RAG) simple développée avec [LlamaIndex](https://github.com/run-llama/llama_index). Elle comporte les étapes essentielles de (i) chargement des documents, (ii) indexation des documents, (iii) récupération des documents en rapport avec la question, et (iv) soumission de la question et des documents pertinants au llm pour réponse.

Docs:
- https://github.com/run-llama/llama_index
- https://docs.llamaindex.ai/

In [65]:
import sys

# Supprimer les commentaires pour installer

if 'google.colab' in sys.modules:
    !pip3 install -q -U gradio
    !pip3 install -q -U llama-index
    !pip3 install -q -U wikipedia
    !pip3 install -q -U llama-index-readers-wikipedia

In [66]:
import gradio as gr
import wikipedia
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import VectorStoreIndex
from llama_index.core.memory import ChatMemoryBuffer

## Chargement de la Clé pour OpenAI

Il vous faut obtenir d'Open AI une clé pour exécuter ce notebook Jupyter. Vous pouvez consulter [Where do I find my API key?](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key). Ensuite, le chargement se fait soit à partir de l'environnement (fichier `.env`), soit à partir des secrets de Google Colab.

In [67]:
import os

openai_api_key = None
if 'google.colab' in sys.modules:
  from google.colab import userdata
  openai_api_key = userdata.get('OPENAI_API_KEY')
else:
  from dotenv import load_dotenv, find_dotenv
  _ = load_dotenv(find_dotenv()) # read local .env file
  openai_api_key  = os.environ['OPENAI_API_KEY']

## Lecture des documents

In [68]:
wikipedia.set_lang('fr')
# wikipedia.search('Marie-Louise d\'Autriche')
# wikipedia.page('Marie-Louise d\'Autriche').content

reader=WikipediaReader()
documents = reader.load_data(['Napoléon_Ier', 'Joséphine_de_Beauharnais', 'Marie-Louise d\'Autriche'])
len(documents)

3

## Indexation des documents

In [69]:
index = VectorStoreIndex.from_documents(documents, insert_batch_size=512)

## Question sur les documents

In [70]:
query_engine = index.as_query_engine()
response = query_engine.query('Quand Napoléon 1er est-il né?')
print(response)

Napoléon 1er est né le 15 août 1769.


## Création d'une application Chat

In [71]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3000)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "Tu es un professeur d'histoire. Tu réponds en Français aux questions sur le seul fondement des documents qui te sont fournis."
        "Quand tu ne trouves pas la répone dans ces documents, tu réponds que tu ne sais pas, même après plusieurs essais."
    ),
)

def simple_chat(message, history=[]):
    return chat_engine.chat(message).response

def stream_chat(message, history=[]):
    response = ''
    stream = chat_engine.stream_chat(message)
    for token in stream.response_gen:
        response += token
        yield response

simple_chat("Quand Napoléon 1er est-il né?")

'Napoléon 1er est né le 15 août 1769 à Ajaccio.'

In [72]:
demo = gr.ChatInterface(
    fn=stream_chat,
    examples=[
        "Quand Napoléon 1er est-il né?",
        "Quand Napoléon 1er est-il mort?",
        "Où ça?",
        "Quand Jeanne d'Arc est-elle née?"
        ],
    title="Démo LlamaIndex"
).queue()
demo.launch()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.
